In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
#import pymysql  need this if I actaully use the code to talk to a DB
import configparser

In [2]:
tripData1 = pd.read_csv('/dltraining/datasets/Trip Data/trip_data_1.csv',
                        nrows=1000)

In [3]:
tripData1.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750660
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770,-74.009834,40.726002
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.976250,40.748528,-74.002586,40.747868


In [4]:
tripFare1 = pd.read_csv('/dltraining/datasets/Trip Fare/trip_fare_1.csv',
                        nrows=1000)

#remove the leading spaces
tripFare1.columns = [
    'medallion', 'hack_license', 'vendor_id', 'pickup_datetime',
    'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount',
    'tolls_amount', 'total_amount'
]

In [5]:
tripFare1.head()

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,2013-01-01 15:11:48,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-06 00:18:35,CSH,6.0,0.5,0.5,0.0,0.0,7.0
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-05 18:49:41,CSH,5.5,1.0,0.5,0.0,0.0,7.0
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:54:15,CSH,5.0,0.5,0.5,0.0,0.0,6.0
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:25:03,CSH,9.5,0.5,0.5,0.0,0.0,10.5


# Database Connection 

In [6]:
config = configparser.ConfigParser()
config

In [14]:
config.read("/dltraining/New-York-Taxi-Analysis/Secrets/passwords.ini")

['/dltraining/New-York-Taxi-Analysis/Secrets/passwords.ini']

In [8]:
config.sections()

['Database_NYTaxi', 'Database_NYLocations']

In [9]:
config.options('Database_NYTaxi')

['user', 'securitytoken', 'host', 'port', 'dbname']

In [10]:
## Database no longer exists, this is a demo only

# Assigning values to variable
host = config.get('Database_NYTaxi', 'host')
port = config.get('Database_NYTaxi', 'port')
dbname = config.get('Database_NYTaxi', 'dbname')
user = config.get('Database_NYTaxi', 'user')
password = config.get('Database_NYTaxi', 'securitytoken')
print(
    "The Database %s is found at %s on port %s. It has a user of %s, with a password of %s"
    % (dbname, host, port, user, password))

The Database NYCTaxiDB is found at nyctaxi.cq3rwooo9ghy.ap-southeast-2.rds.amazonaws.com on port 3306. It has a user of Administrator, with a password of X5LdUuzaZoY7CvUO7Gmpaw4Hed


In [11]:
config['Database_NYLocations']['securitytoken']

'FyD#*BgD9JdjtwFoiOZOCIr9V3od*Y'

In [25]:
# A different take on accessing values in the same way that python does for values  [section][option]
host = config['Database_NYLocations']['host']
port = config['Database_NYLocations']['port']
dbname = config['Database_NYLocations']['dbname']
user = config['Database_NYLocations']['user']
password = config['Database_NYLocations']['securitytoken']
                      
print(
    "The Database \"%s\" is found at \"%s\" on port %s. \n\n"
    "It has a user of \"%s\" with a password of -> %s \n\n"
    % (dbname, host, port, user, password))

The Database "NYCTaxiLocationsDB" is found at "nyctaxilocations.cq3rwooo9ghy.ap-southeast-2.rds.amazonaws.com" on port 3307. 

It has a user of "Administrator" with a password of -> FyD#*BgD9JdjtwFoiOZOCIr9V3od*Y 




In [26]:
# create a connection to the database
conn = pymysql.connect(host, user=user, port=port, passwd=password, db=dbname)

In [ ]:
# create object to talk to the db
cursorObject = conn.cursor()

In [ ]:
# check to see if there are any tables
sqlQuery = "show tables"
cursorObject.execute(sqlQuery)
for x in cursorObject:
    print(x)

In [ ]:
sqlQuery = "CREATE TABLE tripData(id INT AUTO_INCREMENT PRIMARY KEY, medallion char(32), hack_license char(32), vendor_id varchar(32), rate_code int, store_and_fwd_flag char(5), pickup_datetime datetime, dropoff_datetime datetime , passenger_count int, trip_time_in_secs int, trip_distance float(10,5), pickup_longitude float(12, 7), pickup_latitude float(12, 7), dropoff_longitude float(12, 7), dropoff_latitude float(12, 7))"
cursorObject.execute(sqlQuery)

In [ ]:
sqlQuery = "CREATE TABLE fareData(id INT AUTO_INCREMENT PRIMARY KEY, medallion char(32), hack_license char(32), vendor_id varchar(32), pickup_datetime datetime, payment_type varchar(32), fare_amount float(10,3), surcharge float(10,3), mta_tax float(10,3), tip_amount float(10,3), tolls_amount float(10,3), total_amount float(10,3))"
cursorObject.execute(sqlQuery)

In [ ]:
sqlQuery = "show tables"
cursorObject.execute(sqlQuery)
for x in cursorObject:
    print(x)

In [ ]:
tripFare1.to_sql(con=conn, name='fareData_1', if_exists='append', index=False)

In [ ]:
sqlQuery = "select count(*) from fareData"
cursorObject.execute(sqlQuery)
for x in cursorObject:
    print(x)

In [ ]:
sqlQuery = "ALTER TABLE NYCTaxiDB.fareData AUTO_INCREMENT =1"
cursorObject.execute(sqlQuery)
for x in cursorObject:
    print(x)

In [ ]:
sqlQuery = "SELECT name FROM sqlite_master WHERE type='table' AND name='fareData_1'"
cursorObject.execute(sqlQuery)
for x in cursorObject:
    print(x)

In [ ]:
# close the connection
conn.close()